# Practice changes analysis

Practice codes are the unique identifier used to track prescribing in the Practices Prescribing Data.

We assume that a practice code represents prescribing at a particular location.

This analysis compares `epraccur.csv` data from Feb 2016 and March 2017. The current data is made available [here](https://digital.nhs.uk/organisation-data-service/data-downloads/gp-data).



In [97]:
import pandas as pd
import numpy as np
headers = [
    'code','name','postcode','status_code','setting']
df1 = pd.read_csv("2016_02_epraccur.csv", names=headers, usecols=[0,1,9,12,25])
df2 = pd.read_csv("2017_03_epraccur.csv", names=headers, usecols=[0,1,9,12,25])
df1 = df1.set_index('code') 
df2 = df2.set_index('code')

First, we make a table that joins the data from both dates

In [99]:
joined = df1.join(df2, how='inner',  rsuffix='2017')
joined.head()



,name,postcode,status_code,setting,name2017,postcode2017,status_code2017,setting2017
code,,,,,,,,
A81001,THE DENSHAM SURGERY,TS18 1HU,A,4,THE DENSHAM SURGERY,TS18 1HU,A,4
A81002,QUEENS PARK MEDICAL CENTRE,TS18 2AW,A,4,QUEENS PARK MEDICAL CENTRE,TS18 2AW,A,4
A81003,VICTORIA MEDICAL PRACTICE,TS26 8DB,A,4,VICTORIA MEDICAL PRACTICE,TS26 8DB,D,4
A81004,WOODLANDS ROAD SURGERY,TS1 3BE,A,4,WOODLANDS ROAD SURGERY,TS1 3BE,A,4
A81005,SPRINGWOOD SURGERY,TS14 7DJ,A,4,SPRINGWOOD SURGERY,TS14 7DJ,A,4


Now let's focus on items whose status has changed from closed to something else

In [103]:
changed_from_closed = joined[(joined.status_code != joined.status_code2017) & (joined.status_code == 'C')]
print "%s practices went from closed to something else" % changed_from_closed.status_code.count()
#changed[(changed.setting != changed.setting2017) & (changed.status_code2017 != 'C') & ((changed.setting == 4)|(changed.setting2017 == 4))]

0 practices went from closed to something else


In [105]:
changed_from_dormant = joined[(joined.status_code != joined.status_code2017) & (joined.status_code == 'D')]
print "%s practices went from dormant to something else" % changed_from_dormant.status_code.count()
#changed[(changed.setting != changed.setting2017) & (changed.status_code2017 != 'C') & ((changed.setting == 4)|(changed.setting2017 == 4))]

321 practices went from dormant to something else


In [107]:
changed_from_dormant.head()

,name,postcode,status_code,setting,name2017,postcode2017,status_code2017,setting2017
code,,,,,,,,
A81069,RAINBOW SURGERY,TS10 4NW,D,4,RAINBOW SURGERY,TS10 4NW,C,4
A82011,HARTINGTON STREET MEDICAL PRACTICE,LA14 5SL,D,4,HARTINGTON STREET MEDICAL PRACTICE,LA14 5SL,C,4
A82619,NELSON STREET SURGERY,LA15 8AF,D,4,NELSON STREET SURGERY,LA15 8AF,C,4
A82635,GROSVENOR HOUSE SURGERY (W),CA1 1LB,D,4,GROSVENOR HOUSE SURGERY (W),CA1 1LB,C,4
A82641,GROSVENOR HOUSE SURGERY (F),CA1 1LB,D,4,GROSVENOR HOUSE SURGERY (F),CA1 1LB,C,4


In [108]:
# Looks like a lot of these seemed to change their names?

print "%s of these practices kept the same name" % changed_from_dormant[changed_from_dormant.name == changed_from_dormant.name2017].status_code.count()

321 of these practices kept the same name


In [110]:
changed_from_dormant[changed_from_dormant.status_code2017 != 'C']

,name,postcode,status_code,setting,name2017,postcode2017,status_code2017,setting2017
code,,,,,,,,
Y02514,LITHERLAND PRIMARY CARE WALK-IN SERVICE,L21 9JN,D,4,LITHERLAND PRIMARY CARE WALK-IN SERVICE,L21 9JN,A,4
Y02823,DMC VICARAGE LANE,E15 4ES,D,4,DMC VICARAGE LANE,E15 4ES,A,4
